In [14]:
!nvidia-smi

Thu Nov 12 13:40:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install keras-tcn

     |████████████████████████████████| 378kB 8.7MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, spatial, stats
from fastdtw import fastdtw
from scipy.signal import find_peaks
import pywt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from tcn import TCN, tcn_full_summary

In [20]:
os.chdir('gdrive/My Drive/Weave/Earables Project/Final Experiments/')

FileNotFoundError: ignored

In [21]:
data = pickle.load(open('data/processed_df.pkl', 'rb'))

In [22]:
labels = data.iloc[:,-3:-2]
ohe = OneHotEncoder()
ohe.fit(labels)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [8]:
subjects

NameError: ignored

In [23]:
subjects = list(np.unique(data.subject))
# print(subjects)
test_i = 8
test_sub = subjects[test_i]

sub_data = data[data.subject == test_sub]
idx = list(sub_data.index)
indices = np.arange(idx[0][0], idx[-1][0]+1)
np.random.shuffle(indices)

k=5
folds = []
fold_len = len(indices)//k
for i in range(k):
    st = i * fold_len
    en = (i+1) * fold_len 
    fold_i = indices[st:en]
    fold_d = sub_data.loc[fold_i]
    folds.append(fold_d)

In [28]:
test_fold = 0
test_data = folds[test_fold]

train_data = []
for fold_i, fold_d in enumerate([folds[0], folds[1], folds[2]]):
    if fold_i != test_fold:
        train_data.append(fold_d)

train_data = pd.concat(train_data)

# train_data = folds[train_fold]

X_train = train_data.iloc[:,:-3].to_numpy()
y_train = train_data.iloc[:,-3:-2]
X_test = test_data.iloc[:,:-3].to_numpy()
y_test = test_data.iloc[:,-3:-2]
y_train = ohe.transform(y_train).toarray()
y_test = ohe.transform(y_test).toarray()

tmp = []
for i in range(len(X_train)):
    ch = []
    for j in range(12):
        ch.append(X_train[i][j])
    ch = np.array(ch).T
    tmp.append(ch)
X_train = np.array(tmp)

tmp = []
for i in range(len(X_test)):
    ch = []
    for j in range(12):
        ch.append(X_test[i][j])
    ch = np.array(ch).T
    tmp.append(ch)
X_test = np.array(tmp)
del tmp, ch
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4340, 100, 12) (4340, 32)
(2170, 100, 12) (2170, 32)


In [ ]:
n_classes = 32
batch_size = 32
input_shape = (100, 12)

img_input = Input(shape=input_shape)
x = TCN(return_sequences=False, nb_filters=64)(img_input)
# x = TCN(return_sequences=False, nb_filters=128)(x)
x = Dense(n_classes, activation='sigmoid', name='output')(x)

model = Model(img_input, x, name='tcn_adaptive_sub_{}_fold_{}'.format(test_sub, test_fold+1))
# model.summary()

# -----------------------------------------------
model_name = 'tcn_adaptive_sub_{}_fold_{}'.format(test_sub, test_fold+1)
optimizer = Adam(learning_rate=0.001)
# loss = BinaryCrossentropy()
# model.load_weights('checkpoint/{}.h5'.format(model_name))
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

checkpoint = ModelCheckpoint('checkpoint/per_user_accuracy/{}.h5'.format(model_name), monitor='val_accuracy', verbose=1, mode='max',save_best_only=True)
earlystopping = EarlyStopping(monitor='val_accuracy', patience=50, verbose=1)
reducelr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=10, verbose=1, min_lr=0.000001)

# ----------------------------------------------
print('started training...')
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=batch_size,
                    epochs=300,
                    # workers=4,
                    callbacks=[checkpoint, earlystopping],
                    shuffle=True,
                    verbose=1)

# -----------------------------------------------
labels = ohe.categories_[0]
model.load_weights('checkpoint/per_user_accuracy/{}.h5'.format(model_name))
print('Model loaded: checkpoint/per_user_accuracy/{}.h5'.format(model_name))
pred = model.predict(X_test)
y_pred = [labels[i] for i in np.argmax(pred, axis=1)]
y_true = [labels[i] for i in np.argmax(y_test, axis=1)]
print(accuracy_score(y_true, y_pred))

In [ ]:
for test_fold in range(3):
    test_data = folds[test_fold]
    train_data = []
    for fold_i, fold_d in enumerate(folds):
        if fold_i != test_fold:
            train_data.append(fold_d)

    train_data = pd.concat(train_data)

    X_train = train_data.iloc[:,:-3].to_numpy()
    y_train = train_data.iloc[:,-3:-2]
    X_test = test_data.iloc[:,:-3].to_numpy()
    y_test = test_data.iloc[:,-3:-2]
    y_train = ohe.transform(y_train).toarray()
    y_test = ohe.transform(y_test).toarray()

    #--------------------------------------------
    tmp = []
    for i in range(len(X_train)):
        ch = []
        for j in range(12):
            ch.append(X_train[i][j])
        ch = np.array(ch).T
        tmp.append(ch)
    X_train = np.array(tmp)

    tmp = []
    for i in range(len(X_test)):
        ch = []
        for j in range(12):
            ch.append(X_test[i][j])
        ch = np.array(ch).T
        tmp.append(ch)
    X_test = np.array(tmp)
    del tmp, ch
    # print(X_train.shape, y_train.shape)
    # print(X_test.shape, y_test.shape)

    # --------------------------------------
    batch_size = 32
    input_shape = (100, 12)

    img_input = Input(shape=input_shape)
    x = TCN(return_sequences=False, nb_filters=64)(img_input)
    # x = TCN(return_sequences=False, nb_filters=128)(x)
    x = Dense(n_classes, activation='sigmoid', name='output')(x)

    model = Model(img_input, x, name='tcn_sub_{}_fold_{}'.format(test_sub, test_fold+1))
    # model.summary()

    # -----------------------------------------------
    model_name = 'tcn_sub_{}_fold_{}'.format(test_sub, test_fold+1)
    optimizer = Adam(learning_rate=0.001)
    # loss = BinaryCrossentropy()
    # model.load_weights('checkpoint/{}.h5'.format(model_name))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

    checkpoint = ModelCheckpoint('checkpoint/per_user_accuracy/{}.h5'.format(model_name), monitor='val_loss', verbose=0, mode='min',save_best_only=True)
    earlystopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0)
    reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, min_lr=0.000001)

    # ----------------------------------------------
    print('started training...')
    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        batch_size=batch_size,
                        epochs=100,
                        # workers=4,
                        callbacks=[checkpoint, earlystopping],
                        shuffle=True,
                        verbose=0)

    # -----------------------------------------------
    labels = ohe.categories_[0]
    model.load_weights('checkpoint/per_user_accuracy/{}.h5'.format(model_name))
    print('Model loaded: checkpoint/per_user_accuracy/{}.h5'.format(model_name))
    pred = model.predict(X_test)
    y_pred = [labels[i] for i in np.argmax(pred, axis=1)]
    y_true = [labels[i] for i in np.argmax(y_test, axis=1)]
    print(accuracy_score(y_true, y_pred))

started training...
Model loaded: checkpoint/per_user_accuracy/tcn_sub_7_fold_1.h5
0.9502304147465438
started training...
Model loaded: checkpoint/per_user_accuracy/tcn_sub_7_fold_2.h5
0.9447004608294931
started training...
Model loaded: checkpoint/per_user_accuracy/tcn_sub_7_fold_3.h5
0.9414746543778801


In [ ]:
tmp = []
for i in range(len(X_train)):
    ch = []
    for j in range(12):
        ch.append(X_train[i][j])
    ch = np.array(ch).T
    tmp.append(ch)
X_train = np.array(tmp)

tmp = []
for i in range(len(X_test)):
    ch = []
    for j in range(12):
        ch.append(X_test[i][j])
    ch = np.array(ch).T
    tmp.append(ch)
X_test = np.array(tmp)
del tmp, ch
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8680, 100, 12) (8680, 32)
(2170, 100, 12) (2170, 32)


In [ ]:
n_classes = y_train.shape[1]
print('num. of classes: {}'.format(n_classes))

num. of classes: 32


In [ ]:
batch_size = 32
input_shape = (100, 12)

img_input = Input(shape=input_shape)
x = TCN(return_sequences=False, nb_filters=64)(img_input)
# x = TCN(return_sequences=False, nb_filters=128)(x)
x = Dense(n_classes, activation='sigmoid', name='output')(x)

model = Model(img_input, x, name='tcn_sub_{}_fold_{}'.format(test_sub, test_fold+1))
model.summary()

Model: "tcn_sub_7_fold_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 100, 12)]         0         
_________________________________________________________________
tcn_5 (TCN)                  (None, 64)                93248     
_________________________________________________________________
output (Dense)               (None, 32)                2080      
Total params: 95,328
Trainable params: 95,328
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_name = 'tcn_sub_{}_fold_{}'.format(test_sub, test_fold+1)
optimizer = Adam(learning_rate=0.001)
# loss = BinaryCrossentropy()
# model.load_weights('checkpoint/{}.h5'.format(model_name))
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

checkpoint = ModelCheckpoint('checkpoint/per_user_accuracy/{}.h5'.format(model_name), monitor='val_loss', verbose=1, mode='min',save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=0.000001)

In [ ]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=batch_size,
                    epochs=100,
                    # workers=4,
                    callbacks=[checkpoint, earlystopping, reducelr],
                    shuffle=True)

Epoch 1/100
267/272 [============================>.] - ETA: 0s - loss: 0.6233 - accuracy: 0.0730
Epoch 00001: val_loss improved from inf to 0.13807, saving model to checkpoint/per_user_accuracy/tcn_sub_7_fold_3.h5
272/272 [==============================] - 2s 9ms/step - loss: 0.6155 - accuracy: 0.0748 - val_loss: 0.1381 - val_accuracy: 0.1124
Epoch 2/100
266/272 [============================>.] - ETA: 0s - loss: 0.1219 - accuracy: 0.1974
Epoch 00002: val_loss improved from 0.13807 to 0.11730, saving model to checkpoint/per_user_accuracy/tcn_sub_7_fold_3.h5
272/272 [==============================] - 2s 7ms/step - loss: 0.1218 - accuracy: 0.1982 - val_loss: 0.1173 - val_accuracy: 0.2184
Epoch 3/100
266/272 [============================>.] - ETA: 0s - loss: 0.1039 - accuracy: 0.3080
Epoch 00003: val_loss improved from 0.11730 to 0.10555, saving model to checkpoint/per_user_accuracy/tcn_sub_7_fold_3.h5
272/272 [==============================] - 2s 7ms/step - loss: 0.1038 - accuracy: 0.3084

In [ ]:
labels = ohe.categories_[0]
model.load_weights('checkpoint/per_user_accuracy/{}.h5'.format(model_name))
print('Model loaded succesfully!')
pred = model.predict(X_test)
y_pred = [labels[i] for i in np.argmax(pred, axis=1)]
y_true = [labels[i] for i in np.argmax(y_test, axis=1)]
print(classification_report(y_true, y_pred))

Model loaded succesfully!
              precision    recall  f1-score   support

           1       0.96      1.00      0.98        85
          10       0.92      0.95      0.93        80
          11       1.00      0.98      0.99        50
          12       0.83      1.00      0.91        45
          13       0.83      1.00      0.91        85
          14       0.93      0.84      0.88        50
          15       0.85      0.89      0.87        75
          16       1.00      0.96      0.98        70
          17       0.93      0.98      0.96        55
          18       0.97      1.00      0.98        65
          20       0.86      1.00      0.92        90
          22       0.94      0.94      0.94        70
          23       0.96      1.00      0.98        75
          24       1.00      0.98      0.99        65
          25       1.00      0.90      0.95        40
          26       0.96      0.94      0.95        50
          27       1.00      1.00      1.00        80
 